# Load and prepare data
**Objective**: Load news and tweets data from raw file into sqlite3 db.

# Roadmap
1. Copy raw data file to DATA_DIR, unzip and check format.
2. Create db. Build tables for news and tweets.
3. Bulk load news and tweets data into db.
4. Check basic statistics.
4. Build df pickles for news and tweets over selected period

# Steps

In [1]:
"""
Initialization
"""

'''
Standard modules
'''
import os
import sqlite3
import csv
from pprint import pprint

'''
Analysis modules
'''
import pandas as pd

'''
Custom modules
'''
import config
import utilities

'''
Misc
'''
nb_name = '20171002-daheng-load_and_prepare_data'

## Copy raw data file to DATA_DIR, unzip and check format 

Raw data are located inside raw-news_tweets-meng/ folder in DATA_DIR.  

In [9]:
! ls -1 ./data/raw-news_tweets-meng/

dataset1
dataset2
readme.txt
volume.txt


In [10]:
! ls -1 ./data/raw-news_tweets-meng/dataset1

news.txt
tweets.txt


### Check format of news data:

In [12]:
! head -2 ./data/raw-news_tweets-meng/dataset1/news.txt

POST_TIME	NEWS_SOURCE	NEWS_TITLE	NEWS_DOC
2014-11-18 00:03:28	Times of India	India sixth worst affected country by terrorism in 2013: Report	LONDON: India was the sixth worst affected country by terrorism in 2013 — the other five being war zones at present. :::::::::::::::: As many as 43 different terrorist groups categorized into three groups: Islamists, separatists and communists have been found to have planned and carried out attacks in India. :::::::::::::::: The world witnessed 10,000 terrorist attacks in 2013, a 44% increase from the previous year. India, however, witnessed a much higher increase of terrorist attacks during the same period — while terrorism increased by 70% in India from 2012 to 2013, the number of deaths increased from 238 to 404. :::::::::::::::: Globally, the number of deaths from terrorism increased by 61%, according to the Global Terrorism Index 2014. :::::::::::::::: The report says that in India, there remains significant terrorist activity, including on t

### Check format of tweets data:

In [14]:
! head -5 ./data/raw-news_tweets-meng/dataset1/tweets.txt

POST_TIME	TWEET_TEXT
2014-11-10 08:47:19	x + 'Suicide car bomber kills three foreign troops in Afghan capital ...
2014-11-10 14:47:37	x + 'Suicide car bomber kills 3 foreign troops in Afghan capital ...
2014-11-10 15:48:21	Suicide Attack At Cafe In Iraqi Capital Kills 12
2014-11-11 04:44:54	@AlphaTwat yes! 52 at 630am and by 730 it was 18 with blowing snowstorms! This part of the world is crazy.


## Create db. Build tables for news and tweets.

### Define DDL scripts to create db, table schema for news, and table schema for tweets

In [ ]:
"""
Register
    NEWS_TWEETS_DDL_FILE = os.path.join(DATA_DIR, 'news_tweets-meng.schema.sql')
in config.
"""

In [16]:
! cat ./data/news_tweets-meng.schema.sql

-- news_tweets-meng.schema.sql

-- Schema for news and tweets data

-- News table
create table news (
    news_id integer primary key autoincrement not null,
    post_time datetime,
    news_source text,
    news_title text,
    news_doc text
);

-- Tweets table
create table tweets (
    tweet_id integer primary key autoincrement not null,
    post_time datetime,
    tweet_text text
);


### Build db and talbes

In [2]:
"""
Register 
    NEWS_TWEETS_DB = os.path.join(DATA_DIR, 'news_tweets-meng.db')
in config.
"""
def check_db():
    db_exists = os.path.exists(config.NEWS_TWEETS_DB_FILE)
    
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        if not db_exists:
            print('No db exists. Creating new db and tables...')
            with open(config.NEWS_TWEETS_DDL_FILE, 'r') as f:
                schema = f.read()
                conn.executescript(schema)
        else:
            print('db already exists!')

In [3]:
if 1 == 1:
    check_db()

db already exists!


### Bulk load news and tweets into db

In [7]:
%%time
"""
News data

Note: should be executed two times
 - data/raw-news_tweets-meng/dataset1/news.txt
 - data/raw-news_tweets-meng/dataset2/news.txt
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        with open(os.path.join(config.DATA_DIR, 'raw-news_tweets-meng/dataset2/news.txt'), 'r') as f:
            news_df_chunksize = 10000
            for df_chunk in pd.read_csv(f, sep='\t', chunksize=news_df_chunksize, iterator=True):
                column_names = {
                    'POST_TIME': 'post_time',
                    'NEWS_SOURCE': 'news_source',
                    'NEWS_TITLE': 'news_title',
                    'NEWS_DOC': 'news_doc'
                }
                df_chunk = df_chunk.rename(columns=column_names)
                df_chunk.to_sql(name='news', con=conn, if_exists='append', index=False)

CPU times: user 1.38 s, sys: 368 ms, total: 1.75 s
Wall time: 3.32 s


In [5]:
%%time
"""
Tweets data

Note: should be executed two times
 - data/raw-news_tweets-meng/dataset1/tweets.txt
 - data/raw-news_tweets-meng/dataset2/tweets.txt
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        with open(os.path.join(config.DATA_DIR, 'raw-news_tweets-meng/dataset2/tweets.txt'), 'r') as f:
            tweets_df_chunksize = 100000
            for df_chunk in pd.read_csv(f, sep='\t', chunksize=tweets_df_chunksize, iterator=True):
                column_names = {
                    'POST_TIME': 'post_time',
                    'TWEET_TEXT': 'tweet_text'
                }
                df_chunk = df_chunk.rename(columns=column_names)
                df_chunk.to_sql(name='tweets', con=conn, if_exists='append', index=False)

CPU times: user 22.2 s, sys: 1.9 s, total: 24.1 s
Wall time: 29.6 s


In [16]:
%%time
"""
Tweets table is too large. Build index on post_time field.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        query = '''
        create index tweets_post_time on tweets(post_time);
        '''
        cursor.execute(query)

CPU times: user 1min 3s, sys: 9.23 s, total: 1min 12s
Wall time: 1min 23s


In [3]:
"""
List out tables and indices
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        cursor = conn.cursor()
        query = """
        select name from sqlite_master
        where type = 'table';
        """
        cursor.execute(query)
        print('TABLES:')
        print(cursor.fetchall())
        
        query = """
        select name from sqlite_master
        where type = 'index'
        """
        cursor.execute(query)
        print('INDICES:')
        print(cursor.fetchall())

TABLES:
[('news',), ('sqlite_sequence',), ('tweets',)]
INDICES:
[('tweets_post_time',)]


### Check basic statistics

In [2]:
"""
Check number of news per day
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = '''
        select date(post_time) as news_date, count(news_id) as news_num
        from news
        group by date(post_time);
        '''
        
        cursor.execute(query)
        for row in cursor.fetchall():
            print('{}: {}'.format(row['news_date'], row['news_num']))

2014-11-18: 290
2014-11-19: 267
2014-11-20: 320
2014-11-21: 293
2014-11-22: 260
2014-11-23: 275
2014-11-24: 274
2014-11-25: 323
2014-11-26: 307
2014-11-27: 287
2014-11-28: 291
2014-11-29: 250
2014-11-30: 263
2014-12-01: 323
2014-12-02: 291
2014-12-03: 293
2014-12-04: 283
2014-12-05: 288
2014-12-06: 233
2014-12-07: 262
2014-12-08: 292
2014-12-09: 272
2014-12-10: 284
2014-12-11: 284
2014-12-12: 254
2014-12-13: 231
2014-12-14: 149
2014-12-16: 47
2014-12-17: 313
2014-12-18: 51
2014-12-19: 301
2014-12-20: 248
2014-12-21: 268
2014-12-22: 293
2014-12-23: 282
2014-12-24: 287
2014-12-25: 261
2014-12-26: 269
2014-12-27: 253
2014-12-28: 265
2014-12-29: 263
2014-12-30: 69
2014-12-31: 55
2015-01-01: 271
2015-01-02: 282
2015-01-03: 254
2015-01-04: 243
2015-01-05: 262
2015-01-06: 311
2015-01-07: 269
2015-01-08: 275
2015-01-09: 296
2015-01-10: 257
2015-01-11: 282
2015-01-12: 276
2015-01-13: 299
2015-01-14: 311
2015-01-15: 280
2015-01-16: 303
2015-01-17: 248
2015-01-18: 258
2015-01-19: 295
2015-01-20: 

In [25]:
%%time
"""
Check number of tweets per day

Note: date(post_time) function is really slow. Only aggregate on a small subset of tweets data.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        query = """
        select date(post_time) as tweet_date, count(tweet_id) as tweets_num
        from tweets
        where post_time between '2014-11-10' and '2014-11-22'
        group by date(post_time);
        """
        
        cursor.execute(query)
        for row in cursor.fetchall():
            print('{}: {}'.format(row['tweet_date'], row['tweets_num']))

2014-11-10: 3
2014-11-11: 8
2014-11-12: 7
2014-11-13: 12
2014-11-14: 10
2014-11-15: 12
2014-11-16: 24
2014-11-17: 77
2014-11-18: 142824
2014-11-19: 287755
2014-11-20: 324073
2014-11-21: 275824
CPU times: user 1.52 s, sys: 80 ms, total: 1.6 s
Wall time: 1.59 s


### Build df pickles for news and tweets over selected period  
Select out news and tweets data from (3) dataset1: 2015-01-01 to 2015-03-21 according to readme.txt file of raw dataset

In [2]:
%%time
"""
Build pickle for news data over selected period.

Register
    NEWS_PERIOD_DF_PKL = os.path.join(DATA_DIR, 'news-period.df.pkl')
in config.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        query = """
        select news_id, post_time, news_source, news_title, news_doc from news
        where post_time between '2015-01-01' and '2015-03-22';
        """
        
        news_period_df = pd.read_sql_query(sql=query,con=conn,parse_dates=['post_time'])
        
        news_period_df.to_pickle(config.NEWS_PERIOD_DF_PKL)

CPU times: user 568 ms, sys: 560 ms, total: 1.13 s
Wall time: 4.22 s


In [3]:
%%time
"""
Build pickle for tweets data over selected period.

Register
    TWEETS_PERIOD_DF_PKL = os.path.join(DATA_DIR, 'tweets-period.df.pkl')
in config.
"""
if 0 == 1:
    with sqlite3.connect(config.NEWS_TWEETS_DB_FILE) as conn:
        query = """
        select tweet_id, post_time, tweet_text from tweets
        where post_time between '2015-01-01' and '2015-03-22';
        """
        
        tweets_period_df = pd.read_sql_query(sql=query,con=conn,parse_dates=['post_time'])
        
        tweets_period_df.to_pickle(config.TWEETS_PERIOD_DF_PKL)

CPU times: user 2min 29s, sys: 26.4 s, total: 2min 56s
Wall time: 3min 29s


In [3]:
%%time
"""
Test recover df pickle
"""
if 1 == 1:
    news_period_df = pd.read_pickle(config.NEWS_PERIOD_DF_PKL)
    tweets_period_df = pd.read_pickle(config.TWEETS_PERIOD_DF_PKL)

CPU times: user 12.4 s, sys: 6.07 s, total: 18.5 s
Wall time: 31.7 s


In [4]:
news_period_df

,news_id,post_time,news_source,news_title,news_doc
0,10965,2015-01-01 00:03:09,CBS News,"Jeb Bush takes ""natural next step"" toward 2016...",Former Florida governor Jeb Bush (R-FL) addres...
1,10966,2015-01-01 00:03:26,CBC.ca,"Fireworks, parties and prayers usher in 2015",Revellers converged at Nathan Phillips Square ...
2,10967,2015-01-01 00:04:41,ABC News,2 Killed in Helicopter Crash in Southern Arizona,Two people were killed in a helicopter crash W...
3,10968,2015-01-01 00:04:41,The Guardian,Jeb Bush quits board posts ahead of possible W...,Potential Republican presidential candidate Je...
4,10969,2015-01-01 00:06:16,NOLA.com,North Korea's Kim Jong Un to South Korean lead...,"PYONGYANG, North Korea -- North Korean leader ..."
5,10970,2015-01-01 00:08:39,Philly.com,Western states get brutal blast of winter,Ryan Winn bundles up son Ian as they wait for ...
6,10971,2015-01-01 00:09:14,Ynetnews,Abbas paves way to join International Criminal...,Following the Palestinian defeat at the UN Sec...
7,10972,2015-01-01 00:12:44,Washington Post,A look at Egypt's Al-Jazeera English trial,CAIRO — Here is a look at the major events in ...
8,10973,2015-01-01 00:16:27,ABC News,"Storm Brings Snow, Cold to West for New Year's",Bundled-up revelers planning to usher in 2015 ...
9,10974,2015-01-01 00:17:17,MiamiHerald.com,"Storm brings snow, cold to West for New Year's",Bundled-up revelers planning to usher in 2015 ...


In [5]:
news_period_df.dtypes

news_id                 int64
post_time      datetime64[ns]
news_source            object
news_title             object
news_doc               object
dtype: object

In [6]:
tweets_period_df

,tweet_id,post_time,tweet_text
0,14225824,2015-01-01 00:03:51,Stampede kills 35 and injures over 40 in New Y...
1,14225825,2015-01-01 00:04:57,good riddance Doug Marrone is no longer @buffa...
2,14225826,2015-01-01 00:04:59,Any real NFL team should aim higher than Kyle ...
3,14225827,2015-01-01 00:05:03,"“@Dan_Lyons76: Man, Doug Marrone has a spectac..."
4,14225828,2015-01-01 00:05:04,How is Doug Marrone better than Rex Ryan?
5,14225829,2015-01-01 00:05:04,Breaking: Buffalo Bills coach Doug Marrone opt...
6,14225830,2015-01-01 00:05:07,Fuck Doug Marrone. I hope he takes Hackett wit...
7,14225831,2015-01-01 00:05:09,You get rid of Rex Ryan for Doug Marrone?!? Lm...
8,14225832,2015-01-01 00:05:10,"Bills: Owner Terry Pegula says he's ""disappoin..."
9,14225833,2015-01-01 00:05:10,Jets: Team will have strong interest in ex-Bil...


In [7]:
tweets_period_df.dtypes

tweet_id               int64
post_time     datetime64[ns]
tweet_text            object
dtype: object